In [ ]:
# 🔍 Notebook Colab: Confronto visivo tra layer 0 e layer 11 in BERT

!pip install -q transformers matplotlib

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from transformers import TFAutoModel, AutoTokenizer

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name, output_attentions=True)

def visualizza_confronto(frase, head=0):
    inputs = tokenizer(frase, return_tensors="tf")
    outputs = model(**inputs, training=False)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # Prendi le matrici di attenzione dei due layer
    att0 = outputs.attentions[0][0][head].numpy()
    att11 = outputs.attentions[-1][0][head].numpy()

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle(f"Confronto Layer 0 vs Layer 11 - Head {head}", fontsize=16)

    for ax, att, titolo in zip(axes, [att0, att11], ["Layer 0", "Layer 11"]):
        im = ax.imshow(att, cmap="viridis")
        ax.set_title(titolo)
        ax.set_xticks(np.arange(len(tokens)))
        ax.set_yticks(np.arange(len(tokens)))
        ax.set_xticklabels(tokens, rotation=45)
        ax.set_yticklabels(tokens)
        for i in range(len(tokens)):
            for j in range(len(tokens)):
                ax.text(j, i, f"{att[i, j]:.2f}", ha="center", va="center",
                        color="white" if att[i, j] < 0.5 else "black", fontsize=7)
        fig.colorbar(im, ax=ax)

    plt.tight_layout()
    plt.show()

# ✏️ Prova con una frase:
visualizza_confronto("Il gatto mangia il topo")
